# Tradução por Máquina: método estatístico e codificador-decodificador

### Trabalho 3 da disciplina Noções de Inteligência Artificial - 2/2024
### Alunos: Felipe Lopes Gibin Duarte (231025207) e Matheus das Neves Fernandes (231013672)

## Introdução

Neste trabalho vamos usar duas técnicas de tradução automática, a primeira puramente
estatística e a segunda usando ML na arquitetura codificador-decodificador. Vamos analisar a eficácia de ambas, destacando aspectos teóricos importantes.

## 1. Base de Dados

### 1.1 Download da base de dados

Usaremos a base Português - Inglês do parlamento europeu, disponível em https://www.statmt.org/europarl/. Encontre o link “parallel corpus Portuguese-English” e extraia o condeúdo do zip. Pela falta de recursos computacionais, o tamanho dos arquivos foi reduzido para 11 mil sentenças cada.

### 1.2 Upload dos arquivos

In [11]:
#Upload dos arquivos extraídos para o google colab
from google.colab import files

uploaded = files.upload()

Saving reduced_europarl_pt.txt to reduced_europarl_pt (1).txt
Saving reduced_europarl_en.txt to reduced_europarl_en (1).txt


### 1.3 Processamento dos arquivos

In [12]:
# Le os arquivos e remove line breaks
with open('/content/reduced_europarl_pt.txt', 'r', encoding='utf-8') as pt_file:
    portuguese_sentences = [line.strip() for line in pt_file.readlines()]

with open('/content/reduced_europarl_en.txt', 'r', encoding='utf-8') as en_file:
    english_sentences = [line.strip() for line in en_file.readlines()]

In [13]:
# Remove pontuação e converte para letras minusculas
import re
def clean_text(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  return text

# Exibir algumas linhas para verificar
print("Frases em português:", portuguese_sentences[:5])
print("Frases em inglês:", english_sentences[:5])

Frases em português: ['Reinício da sessão', 'Declaro reaberta a sessão do Parlamento Europeu, que tinha sido interrompida na sexta-feira, 17 de Dezembro último, e renovo todos os meus votos, esperando que tenham tido boas férias.', 'Como puderam constatar, o grande "bug do ano 2000" não aconteceu. Em contrapartida, os cidadãos de alguns dos nossos países foram vítimas de catástrofes naturais verdadeiramente terríveis.', 'Os senhores manifestaram o desejo de se proceder a um debate sobre o assunto nos próximos dias, durante este período de sessões.', 'Entretanto, gostaria - como também me foi pedido por um certo número de colegas - que observássemos um minuto de silêncio por todas as vítimas, nomeadamente das tempestades, nos diferentes países da União Europeia que foram afectados.']
Frases em inglês: ['Resumption of the session', 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in th

## 2. IBM Model 1

O IBM Model 1 é um modelo complexo de tradução estatística, amplamente utilizado antes da popularização das traduções baseadas em redes neurais. Ele se baseia em probabilidades de palavras individuais e sua correspondência entre os idiomas, ou seja, o modelo estima a melhor tradução palavra por palavra.

### 2.1 Treinando o modelo IBM Model 1

In [ ]:
#Instalando o nltk
!pip install nltk

import nltk
from nltk.translate import IBMModel1
from nltk.translate import AlignedSent
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab') #Necessario para a tokenização

portuguese_tokens = [word_tokenize(sentence) for sentence in portuguese_sentences]
english_tokens = [word_tokenize(sentence) for sentence in english_sentences]

# Criar o corpus alinhado em pares de listas
parallel_corpus = [AlignedSent(pt, en) for pt, en in zip(portuguese_tokens, english_tokens)]

# Treinar o modelo IBM Model 1
model = IBMModel1(parallel_corpus, 5)  # 5 iterações de treinamento

print("Modelo treinado com sucesso!")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Modelo treinado com sucesso!


### 2.2 Avaliando a qualidade da tradução

In [ ]:
from nltk.translate import IBMModel1, AlignedSent
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Suavização para BLEU
smoothing = SmoothingFunction()

# Função para traduzir uma sentença usando o IBM Model 1
def translate_sentence(model, sentence):
    translated_tokens = []
    for word in sentence:
        # Verificar se a palavra existe na tabela de tradução
        if word in model.translation_table and model.translation_table[word]:
            # Obter a melhor tradução
            best_translation = max(model.translation_table[word], key=model.translation_table[word].get)
        else:
            best_translation = word  # Fallback: palavra original
        translated_tokens.append(best_translation)
    return translated_tokens

# Inicializar referências e hipóteses
references = []
hypotheses = []

# Iterar sobre as sentenças da base
for pt_sentence, en_sentence in zip(portuguese_sentences, english_sentences):
    # Tokenizar as sentenças
    pt_tokens = pt_sentence.split()
    en_tokens = en_sentence.split()

    # Traduzir a sentença em português
    translated_tokens = translate_sentence(model, pt_tokens)

    # Adicionar ao conjunto de referências e hipóteses
    references.append([en_tokens])  # Referência: lista de listas (uma lista por referência)
    hypotheses.append(translated_tokens)  # Hipótese: lista de palavras traduzidas

# Calcular o BLEU score para cada sentença
bleu_scores = [
    sentence_bleu(ref, hyp, smoothing_function=smoothing.method1)
    for ref, hyp in zip(references, hypotheses)
]

# Calcular a média do BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Média do BLEU score para todas as sentenças: {average_bleu_score:.4f}")


Média do BLEU score para todas as sentenças: 0.0206


### 2.3 Considerações sobre o BLEU score

O BLEU score (Bilingual Evaluation Understudy) é uma métrica usada para avaliar a qualidade de traduções automáticas, comparando a tradução gerada pelo modelo com uma ou mais traduções de referência. No caso do IBM model 1, o resultado encontrado foi desapontador devido à abordagem de tradução palavra a palavra, que é ineficiente. Alem disso, a limitação em tamanho da base de dados devido à restrições de capacidade computacional foi outro fator relevante para o score médio ter sido tão baixo (0.0206).  

## 3. Codificador-Decodificador

### 3.1 Upload e processamento dos dados

In [14]:
# Exibir algumas linhas para verificar
print("Frases em português:", portuguese_sentences[:5])
print("Frases em inglês:", english_sentences[:5])

Frases em português: ['Reinício da sessão', 'Declaro reaberta a sessão do Parlamento Europeu, que tinha sido interrompida na sexta-feira, 17 de Dezembro último, e renovo todos os meus votos, esperando que tenham tido boas férias.', 'Como puderam constatar, o grande "bug do ano 2000" não aconteceu. Em contrapartida, os cidadãos de alguns dos nossos países foram vítimas de catástrofes naturais verdadeiramente terríveis.', 'Os senhores manifestaram o desejo de se proceder a um debate sobre o assunto nos próximos dias, durante este período de sessões.', 'Entretanto, gostaria - como também me foi pedido por um certo número de colegas - que observássemos um minuto de silêncio por todas as vítimas, nomeadamente das tempestades, nos diferentes países da União Europeia que foram afectados.']
Frases em inglês: ['Resumption of the session', 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in th

### 3.2 Importações e configurações


In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Configurações
max_vocab_size = 25000  # Tamanho máximo do vocabulário
max_seq_length = 30     # Tamanho máximo da sequência
embedding_dim = 256     # Dimensão do embedding
latent_dim = 512        # Dimensão do vetor latente (de contexto) do RNN
batch_size = 64
epochs = 20

### 3.3 Tokenização e padding

A tokenização converte palavras em números, onde cada número representa o índice de uma palavra no vocabulário. Palavras associadas a índices de número mais baixo são mais frequentes no texto. O token "start" e "end" indicam onde a tradução começa e onde termina. O padding garante que todas as sentenças tenham o mesmo comprimento, se for uma frase mais curta que max__seq_length, serão adicionados zeros ao final das frases.

In [18]:
#Adicionamos tokens de inicio e fim em ingles
english_sentences = ["<start> " + sentence.strip() + " <end>" for sentence in english_sentences]
print(english_sentences[1])

<start> I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period. <end>


In [21]:
#tokenização
tokenizer_pt = Tokenizer(num_words=max_vocab_size)
tokenizer_pt.fit_on_texts(portuguese_sentences)

tokenizer_en = Tokenizer(num_words=max_vocab_size)
tokenizer_en.fit_on_texts(english_sentences)

In [22]:
# Sequências numericas das sentenças em portugues e ingles
input_sequences = tokenizer_pt.texts_to_sequences(portuguese_sentences)
target_sequences = tokenizer_en.texts_to_sequences(english_sentences)

In [23]:
#Adiciona padding ao final das sentenças, de forma a padronizar o tamanho delas
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_length, padding="post")
target_sequences = pad_sequences(target_sequences, maxlen=max_seq_length, padding="post")


In [25]:
#Criação dos arrays de entrada e saida
encoder_input_data = np.array(input_sequences)
decoder_input_data = np.array(target_sequences[:, :-1])  # Sem o token <end>
decoder_target_data = np.array(target_sequences[:, 1:])  # Sem o token <start>, usado pra prever a próxima palavra

### 3.4 Estrutura do modelo encoder-decoder

O encoder processa a sequência de entrada (sentenças em português) e gera um vetor de contexto que contém a representação da sequência. O vetor de contexto é então passado para o decodificador. Neste caso, encoder_outputs pode ser ignorado, só estamos interessados no vetor state_h.

In [24]:
# Definindo o encoder

# A entrada é uma sequência de índices (tokens) representando palavras em português
encoder_inputs = Input(shape=(max_seq_length,))

#Embedding para representar cada palavra como um vetor
encoder_embedding = Embedding(input_dim=max_vocab_size, output_dim=embedding_dim)(encoder_inputs)

#RNN que vai gerar o vetor de estado
encoder_rnn = SimpleRNN(latent_dim, return_state=True)
encoder_outputs, state_h = encoder_rnn(encoder_embedding)

O decoder usa o estado latente (state_h) gerado pelo encoder para produzir a tradução. Ele precisa ser alimentado com o token "start" no início, gerando uma palavra por vez até encontrar o token "end".

In [ ]:
# Definindo o decoder

#Sequencia de tokens em ingles, sem o <end>
decoder_inputs = Input(shape=(max_seq_length,))

#Embedding dos tokens em ingles
decoder_embedding = Embedding(input_dim=max_vocab_size, output_dim=embedding_dim)(decoder_inputs)

#RNN do decoder
#decoder_outputs: Saída para cada passo temporal da RNN no decoder.
#A RNN usa o estado final do encoder (state_h) como estado inicial.
decoder_rnn = SimpleRNN(latent_dim, return_sequences=True)
decoder_outputs = decoder_rnn(decoder_embedding, initial_state=state_h)

#Camada densa para prever a próxima palavra
# A softmax prevê a próxima palavra em inglês como uma probabilidade sobre o vocabulário
decoder_dense = Dense(max_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

Agora, combinamos o encoder e o decoder para montar o modelo completo que mapeará as frases em ingles para as suas traduções em inglês.

In [ ]:
#O modelo recebe como entrada encoder_inputs e decoder_inputs e tem como saida decoder_outputs (frases em ingles traduzidas)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

### 3.5 Treinamento

In [ ]:
#Compilação do modelo completo

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
#Treinamento do modelo

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2  # 20% para validação.
)

### 3.6 Tradução de frases simples

A arquitetura que criamos é para o treinamento. Para usar o modelo treinado na prática, precisamos configurar o encoder e decoder para inferência. Na inferência (tradução), o processo é diferente porque não temos a frase-alvo completa (em inglês). A tradução precisa ser gerada uma palavra por vez, de forma iterativa.

In [ ]:
#Encoder para inferencia
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
#Decoder para inferencia
#Recebe os estados latentes do encoder como entrada e gera palavras passo a passo

# Entradas para os estados iniciais do decoder
decoder_state_input = Input(shape=(latent_dim,))

# Entrada da palavra atual
decoder_single_input = Input(shape=(1,))  # Entrada com uma palavra por vez

# Embedding para a palavra atual
decoder_embedded = decoder_embedding(decoder_single_input)

# RNN usando o estado anterior
decoder_output, decoder_state = decoder_rnn(decoder_embedded, initial_state=decoder_state_input)

# Camada densa para prever a próxima palavra
decoder_prediction = decoder_dense(decoder_output)

# Modelo do decoder para inferência
decoder_model = Model(
    [decoder_single_input, decoder_state_input],  # Entradas: palavra + estado anterior
    [decoder_prediction, decoder_state]          # Saídas: previsão + estado atual
)

Criaremos uma função de tradução que passa uma frase em portugûes e retorna a frase traduzida para o inglês. Após isso, traduziremos uma frase simples

In [ ]:
def translate_sentence(input_sequence, encoder_model, decoder_model, tokenizer_en, max_seq_length):
    # Obtenha o estado latente do encoder
    encoder_states = encoder_model.predict(input_sequence)

    # Comece a tradução com o token <start>
    target_sequence = np.zeros((1, 1))  # Tamanho (1 frase, 1 palavra)
    target_sequence[0, 0] = tokenizer_en.word_index["<start>"]

    decoded_sentence = []

    # Iterar para gerar palavras até o token <end> ou o limite de comprimento
    for _ in range(max_seq_length):
        # Previsão da próxima palavra
        output_tokens, h = decoder_model.predict([target_sequence] + [encoder_states])

        # Palavra com maior probabilidade
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_en.index_word[sampled_token_index]

        # Adiciona a palavra à sentença decodificada
        if sampled_word == "<end>":
            break
        decoded_sentence.append(sampled_word)

        # Atualize a sequência de entrada para o próximo passo
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index

        # Atualize os estados do decoder
        encoder_states = h

    return " ".join(decoded_sentence)


In [ ]:
# Teste de uma frase simples

### 3.7 Avaliação

Avaliaremos o modelo usando o BLEU score, novamente. Espera-se que a tradução seja mais eficiente pelo fato de RNNs serem capazes de capturar o contexto das frases, gerando traduções mais naturais e eficientes.